In [81]:
import os
import json
from glob import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler

In [82]:
data = pd.read_csv("Duolingo Data/duolingo_train_data_clean.csv")
data

,block_id,token_id,token,pos,features,dep_rel,dep_parent,correctness,user_id,format,prompt,countries,days,client,session,time
0,block_1,8XTyQUAl0101,Le,DET,Definite=Def|Gender=Masc|Number=Sing|fPOS=DET++,det,2,0,YjS/mQOx,reverse_translate,The boy,CA,0.005,web,lesson,14.0
1,block_1,8XTyQUAl0102,garçon,NOUN,Gender=Masc|Number=Sing|fPOS=NOUN++,ROOT,0,0,YjS/mQOx,reverse_translate,The boy,CA,0.005,web,lesson,14.0
2,block_2,8XTyQUAl0201,Je,PRON,Number=Sing|Person=1|PronType=Prs|fPOS=PRON++,nsubj,4,0,YjS/mQOx,reverse_translate,I am a woman.,CA,0.005,web,lesson,14.0
3,block_2,8XTyQUAl0202,suis,VERB,Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbF...,cop,4,0,YjS/mQOx,reverse_translate,I am a woman.,CA,0.005,web,lesson,14.0
4,block_2,8XTyQUAl0203,une,DET,Definite=Ind|Gender=Fem|Number=Sing|PronType=D...,det,4,0,YjS/mQOx,reverse_translate,I am a woman.,CA,0.005,web,lesson,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243977,block_85319,93hqXjPO0202,journée,NOUN,Gender=Fem|Number=Sing|fPOS=NOUN++,ROOT,0,0,a3jLlbgc,reverse_translate,Have a good day!,US,15.717,ios,practice,7.0
243978,block_85320,93hqXjPO0301,Le,DET,Definite=Def|Gender=Masc|Number=Sing|fPOS=DET++,det,2,0,a3jLlbgc,reverse_translate,The month of March,US,15.717,ios,practice,5.0
243979,block_85320,93hqXjPO0302,mois,NOUN,Gender=Masc|Number=Sing|fPOS=NOUN++,ROOT,0,1,a3jLlbgc,reverse_translate,The month of March,US,15.717,ios,practice,5.0
243980,block_85320,93hqXjPO0303,de,ADP,fPOS=ADP++,case,4,0,a3jLlbgc,reverse_translate,The month of March,US,15.717,ios,practice,5.0


# Feature Engineering

## Time-Based Features

In [83]:
# Churn Time - time difference between user's last and first activity
data['churn_time'] = data.groupby('user_id')['days'].transform(lambda x: x.max() - x.min())
data['churn_time'] = data['churn_time'].apply(np.floor)

# Time difference between consecutive activities
def assign_session(group):
    # Sort the group by 'days'
    group = group.sort_values("days").copy()
    # Calculate the time difference between consecutive rows
    group['time_diff'] = group['days'].diff().fillna(0)
    # A new session starts if the difference exceeds 0.0417 (approx. 1 hour)
    group['session_id'] = (group['time_diff'] > 0.0417).cumsum() + 1
    return group

data = data.groupby(['user_id'], group_keys=False).apply(assign_session)

# Churn time category (binned version of churn_time)
data['churn_time_category'] = pd.cut(data['churn_time'], 
                                    bins=[0, 6, 13, 20, 1000], 
                                    labels=[1, 2, 3, 4],
                                    include_lowest=True)

/var/folders/_l/sqkjh1g90ls18nynpcqgbk_h0000gn/T/ipykernel_80591/761110689.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby(['user_id'], group_keys=False).apply(assign_session)


## Correctness Features

In [84]:
# Block average correctness
data['block_average_correctness'] = data.groupby(['user_id', 'block_id'])['correctness'].transform('mean')

# Student average correctness (across all blocks)
student_data = data.groupby(['user_id', 'block_id'])['block_average_correctness'].mean().reset_index()
student_data = student_data.groupby(['user_id'])['block_average_correctness'].mean().reset_index()
## student_data.rename(columns={"block_average_correctness": "student_average_correctness"}, inplace=True)

# Token correctness - average correctness for each token
# data['token_correctness'] = data.groupby('token')['correctness'].transform('mean')

# Format-specific correctness
# Reverse tap average correctness
reverse_tap_correctness = data[data['format'] == 'reverse_tap'].groupby('user_id')['correctness'].mean().reset_index()
reverse_tap_correctness = reverse_tap_correctness.rename(columns={'correctness': 'reverse_tap_average_correctness'})

# Reverse translate average correctness
reverse_translate_correctness = data[data['format'] == 'reverse_translate'].groupby('user_id')['correctness'].mean().reset_index()
reverse_translate_correctness = reverse_translate_correctness.rename(columns={'correctness': 'reverse_translate_average_correctness'})

# Listen average correctness
listen_correctness = data[data['format'] == 'listen'].groupby('user_id')['correctness'].mean().reset_index()
listen_correctness = listen_correctness.rename(columns={'correctness': 'listen_average_correctness'})

## Response Time Features

In [85]:
# Response time variance
variance_df = data.groupby(['user_id', 'block_id'])['time'].mean().reset_index()
variance_df = variance_df.groupby('user_id')['time'].var().reset_index()
variance_df = variance_df.rename(columns={'time': 'response_time_variance'})
data = pd.merge(data, variance_df, on='user_id', how='left')

# Block average response time
block_avg_response_time = data.groupby(['user_id', 'block_id'])['time'].mean().reset_index()
block_avg_response_time_per_user = block_avg_response_time.groupby('user_id')['time'].mean().reset_index()
block_avg_response_time_per_user = block_avg_response_time_per_user.rename(columns={'time': 'block_average_response_time'})

# Format-specific average response times
# Reverse tap average response time
reverse_tap_response_time = data[data['format'] == 'reverse_tap'].groupby('user_id')['time'].mean().reset_index()
reverse_tap_response_time = reverse_tap_response_time.rename(columns={'time': 'reverse_tap_average_response_time'})

# Reverse translate average response time
reverse_translate_response_time = data[data['format'] == 'reverse_translate'].groupby('user_id')['time'].mean().reset_index()
reverse_translate_response_time = reverse_translate_response_time.rename(columns={'time': 'reverse_translate_average_response_time'})

# Listen average response time
listen_response_time = data[data['format'] == 'listen'].groupby('user_id')['time'].mean().reset_index()
listen_response_time = listen_response_time.rename(columns={'time': 'listen_average_response_time'})

## Token Easiness

In [86]:
# Block-wise token easiness - average token correctness for each block
# data["block_wise_token_easiness"] = data.groupby(['block_id'])['token_correctness'].transform('mean')

## Format Counts and Session Features

In [87]:
# Session count - number of unique sessions per user
student_session_data = data.groupby('user_id')['session_id'].nunique().reset_index(name='session_count')

# Block count - number of unique blocks per user
student_block_data = data.groupby('user_id')['block_id'].nunique().reset_index(name='block_count')

# Format counts - number of times each format appears for each user
format_data = data.groupby(['user_id', 'block_id'], as_index=False)['format'].first().reset_index()
format_by_stud = pd.crosstab(format_data['user_id'], format_data['format']).reset_index()

## Ability Features

In [88]:
# Create an ability measure that combines correctness and response time
# Higher correctness and lower response times indicate higher ability
user_avg_correctness = data.groupby('user_id')['correctness'].mean().reset_index()
user_avg_response_time = data.groupby('user_id')['time'].mean().reset_index()

# Normalize response time (lower is better, so we invert it)
max_time = user_avg_response_time['time'].max()
user_avg_response_time['normalized_time'] = 1 - (user_avg_response_time['time'] / max_time)

# Create ability score (simple average of correctness and normalized time)
ability_score = pd.merge(user_avg_correctness, user_avg_response_time, on='user_id')
ability_score['user_ability_score'] = (ability_score['correctness'] + ability_score['normalized_time']) / 2

# Keep only user_id and ability score
ability_score = ability_score[['user_id', 'user_ability_score']]

## Count Features

In [89]:
# Create a discrete day variable from the floating-point 'days' column
data['day'] = data['days'].apply(np.floor)

# Now let's recalculate the count features using the new 'day' variable

# Blocks per session remains the same as it doesn't depend on days
blocks_per_session = data.groupby(['user_id', 'session_id'])['block_id'].nunique().reset_index()
blocks_per_session = blocks_per_session.rename(columns={'block_id': 'blocks_in_session'})

# Average number of blocks per session
avg_blocks_per_session = blocks_per_session.groupby('user_id')['blocks_in_session'].mean().reset_index()
avg_blocks_per_session = avg_blocks_per_session.rename(columns={'blocks_in_session': 'average_blocks_per_session'})

# Now use the discrete 'day' variable for daily counts
# Blocks per day: count unique blocks per user per day
user_days = data.groupby(['user_id', 'day'])['block_id'].nunique().reset_index()
user_days = user_days.rename(columns={'block_id': 'blocks_in_day'})

# Average number of blocks per day for each user
avg_blocks_per_day = user_days.groupby('user_id')['blocks_in_day'].mean().reset_index()
avg_blocks_per_day = avg_blocks_per_day.rename(columns={'blocks_in_day': 'average_blocks_per_day'})

# Sessions per day: count unique sessions per user per day
sessions_per_day = data.groupby(['user_id', 'day'])['session_id'].nunique().reset_index()
sessions_per_day = sessions_per_day.rename(columns={'session_id': 'sessions_in_day'})

# Average number of sessions per day for each user
avg_sessions_per_day = sessions_per_day.groupby('user_id')['sessions_in_day'].mean().reset_index()
avg_sessions_per_day = avg_sessions_per_day.rename(columns={'sessions_in_day': 'average_sessions_per_day'})

## Consistency Measures

In [90]:
# Session length variance
session_lengths = data.groupby(['user_id', 'session_id']).size().reset_index(name='session_length')
session_length_variance = session_lengths.groupby('user_id')['session_length'].var().reset_index()
session_length_variance = session_length_variance.rename(columns={'session_length': 'session_length_variance'})

# Variance of number of blocks per session
blocks_per_session_var = blocks_per_session.groupby('user_id')['blocks_in_session'].var().reset_index()
blocks_per_session_var = blocks_per_session_var.rename(columns={'blocks_in_session': 'blocks_per_session_variance'})

# Variance of number of blocks per day
blocks_per_day_var = user_days.groupby('user_id')['blocks_in_day'].var().reset_index()
blocks_per_day_var = blocks_per_day_var.rename(columns={'blocks_in_day': 'blocks_per_day_variance'})

# Variance of number of sessions per day
sessions_per_day_var = sessions_per_day.groupby('user_id')['sessions_in_day'].var().reset_index()
sessions_per_day_var = sessions_per_day_var.rename(columns={'sessions_in_day': 'sessions_per_day_variance'})

# Final User-Level Dataset

In [91]:
# Merge all the user-level features
user_features = pd.merge(student_data, reverse_tap_correctness, on='user_id', how='left')
user_features = pd.merge(user_features, reverse_translate_correctness, on='user_id', how='left')
user_features = pd.merge(user_features, listen_correctness, on='user_id', how='left')
user_features = pd.merge(user_features, block_avg_response_time_per_user, on='user_id', how='left')
user_features = pd.merge(user_features, reverse_tap_response_time, on='user_id', how='left')
user_features = pd.merge(user_features, reverse_translate_response_time, on='user_id', how='left')
user_features = pd.merge(user_features, listen_response_time, on='user_id', how='left')
user_features = pd.merge(user_features, ability_score, on='user_id', how='left')
user_features = pd.merge(user_features, avg_blocks_per_session, on='user_id', how='left')
user_features = pd.merge(user_features, avg_blocks_per_day, on='user_id', how='left')
user_features = pd.merge(user_features, avg_sessions_per_day, on='user_id', how='left')
user_features = pd.merge(user_features, session_length_variance, on='user_id', how='left')
user_features = pd.merge(user_features, blocks_per_session_var, on='user_id', how='left')
user_features = pd.merge(user_features, blocks_per_day_var, on='user_id', how='left')
user_features = pd.merge(user_features, sessions_per_day_var, on='user_id', how='left')
user_features = pd.merge(user_features, student_session_data, on='user_id', how='left')
user_features = pd.merge(user_features, student_block_data, on='user_id', how='left')
user_features = pd.merge(user_features, format_by_stud, on='user_id', how='left')
user_features = pd.merge(user_features, variance_df, on='user_id', how='left')

# Add churn time related features
churn_data = data.groupby("user_id")[["churn_time", "churn_time_category"]].first().reset_index()
user_features = pd.merge(user_features, churn_data, on='user_id', how='left')

# Impute missing data: Replace NaNs in format-specific metrics with 0
format_specific_columns = [
    'reverse_tap_average_correctness', 'reverse_translate_average_correctness', 'listen_average_correctness',
    'reverse_tap_average_response_time', 'reverse_translate_average_response_time', 'listen_average_response_time'
]
user_features[format_specific_columns] = user_features[format_specific_columns].fillna(0)

# Impute missing data: Replace NaNs in variance metrics with 0
variance_columns = ['blocks_per_day_variance', 'sessions_per_day_variance']
user_features[variance_columns] = user_features[variance_columns].fillna(0)

In [92]:
# Save the final raw dataset
user_features.to_csv("Duolingo_data_03_06_final_raw.csv", index=False)

In [93]:
df_raw = pd.read_csv("Duolingo_data_03_06_final_raw.csv")
pd.set_option('display.max_columns', None)
df_raw

,user_id,block_average_correctness,reverse_tap_average_correctness,reverse_translate_average_correctness,listen_average_correctness,block_average_response_time,reverse_tap_average_response_time,reverse_translate_average_response_time,listen_average_response_time,user_ability_score,average_blocks_per_session,average_blocks_per_day,average_sessions_per_day,session_length_variance,blocks_per_session_variance,blocks_per_day_variance,sessions_per_day_variance,session_count,block_count,listen,reverse_tap,reverse_translate,response_time_variance,churn_time,churn_time_category
0,+4kwmfjD,0.227673,0.133858,0.378947,0.375000,8.138365,6.614173,14.968421,13.763889,0.610942,53.000000,159.000000,3.00,12142.333333,1813.000000,0.000000,0.000000,3,159,27,96,36,48.512380,0.0,1
1,+IUzKF2l,0.332900,0.212815,0.500000,0.607595,8.745455,6.995238,14.187500,17.155844,0.646135,21.000000,21.000000,1.00,604.818182,87.400000,87.400000,0.000000,11,231,37,163,31,60.254545,22.0,4
2,+Xv/6Nop,0.229710,0.140162,0.375000,0.397849,12.295652,9.417790,17.423077,23.688172,0.606883,32.857143,46.000000,1.60,2669.142857,565.142857,1901.500000,0.800000,7,230,40,151,39,1171.204785,5.0,1
3,+mMyHsNU,0.239135,0.000000,0.250307,0.225892,10.729670,0.000000,10.867485,13.401585,0.613978,35.000000,91.000000,2.60,14049.910256,1082.333333,3364.000000,2.300000,13,455,219,0,236,169.677862,6.0,1
4,+ziSnwWC,0.169845,0.065789,0.266409,0.328947,12.036199,5.109649,21.324324,16.921053,0.590596,10.523810,18.416667,1.75,569.661905,103.561905,237.719697,0.931818,21,221,36,107,78,141.807775,13.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,y6797eWc,0.076945,0.015957,0.202454,0.090909,8.955414,3.042553,19.411043,15.272727,0.545205,15.700000,39.250000,2.50,430.177778,26.900000,370.916667,1.666667,10,157,29,84,44,486.017230,3.0,1
308,yka558y8,0.144504,0.103175,0.166065,0.175000,30.436170,3.031746,32.227437,98.987500,0.559939,20.888889,47.000000,2.25,2368.250000,498.861111,6418.666667,3.583333,9,188,36,55,97,54144.632268,4.0,1
309,z/C43yrd,0.069785,0.044248,0.175258,0.088889,10.225806,6.415929,26.010309,14.400000,0.536617,11.071429,22.142857,2.00,850.219780,197.609890,633.142857,1.666667,14,155,23,100,32,108.007122,6.0,1
310,z9FlLLzt,0.122254,0.000000,0.104911,0.105263,13.271676,0.000000,15.243304,12.210526,0.545946,19.222222,43.250000,2.25,1072.861111,124.194444,196.250000,0.250000,9,173,27,0,146,87.326926,3.0,1


In [94]:
# Check outliers in response_time_variance using IQR method
Q1 = df_raw['response_time_variance'].quantile(0.25)
Q3 = df_raw['response_time_variance'].quantile(0.75)
IQR = Q3 - Q1

# Count outliers (beyond 1.5 * IQR)
outliers = df_raw[(df_raw['response_time_variance'] < (Q1 - 1.5 * IQR)) | 
                  (df_raw['response_time_variance'] > (Q3 + 1.5 * IQR))]

print(f"Total outliers: {len(outliers)} out of {len(df_raw)} ({len(outliers)/len(df_raw)*100:.2f}%)")
print(f"Outlier threshold: > {Q3 + 1.5*IQR:.2f}")
print("\nTop 5 outliers:")
print(df_raw['response_time_variance'].nlargest(5))

Total outliers: 63 out of 312 (20.19%)
Outlier threshold: > 1822.86

Top 5 outliers:
88     5.825717e+07
144    4.514999e+07
209    1.470882e+07
205    1.251464e+07
297    1.247615e+07
Name: response_time_variance, dtype: float64


In [95]:
# Standardize all numerical input features
# Categories of features to standardize:

# Define feature groups
variance_features = [
    'session_length_variance', 'blocks_per_session_variance',
    'blocks_per_day_variance', 'sessions_per_day_variance', 'response_time_variance'
]

time_features = [
    'block_average_response_time', 'reverse_tap_average_response_time',
    'reverse_translate_average_response_time', 'listen_average_response_time'
]

count_features = [
    'average_blocks_per_session', 'average_blocks_per_day', 'average_sessions_per_day',
    'session_count', 'block_count', 'listen', 'reverse_tap', 'reverse_translate'
]

# user ability score and correctness features are already standardized when engineered

# 1. Handle regular variance features with log transform
for feature in variance_features:
    user_features[feature] = np.log1p(user_features[feature])

# 2. Handle response_time_variance with QuantileTransformer, given 20% of the data are considered outliers above IQR
qt = QuantileTransformer(output_distribution='uniform')  # uniform keeps it in 0-1 range
user_features['response_time_variance'] = qt.fit_transform(
    user_features[['response_time_variance']])

# 3. Apply MinMaxScaler to all numerical features
all_standardized_features = variance_features + time_features + count_features + ['response_time_variance']

# Print which columns will be standardized for verification
print(f"Columns being standardized ({len(all_standardized_features)}):")
print(all_standardized_features)

min_max_scaler = MinMaxScaler()
user_features[all_standardized_features] = min_max_scaler.fit_transform(user_features[all_standardized_features])


# Print summary statistics to verify standardization worked
print("\nAfter standardization:")
print(user_features[all_standardized_features].describe())

Columns being standardized (18):
['session_length_variance', 'blocks_per_session_variance', 'blocks_per_day_variance', 'sessions_per_day_variance', 'response_time_variance', 'block_average_response_time', 'reverse_tap_average_response_time', 'reverse_translate_average_response_time', 'listen_average_response_time', 'average_blocks_per_session', 'average_blocks_per_day', 'average_sessions_per_day', 'session_count', 'block_count', 'listen', 'reverse_tap', 'reverse_translate', 'response_time_variance']

After standardization:
       session_length_variance  blocks_per_session_variance  \
count               312.000000                   312.000000   
mean                  0.471677                     0.498884   
std                   0.175729                     0.171920   
min                   0.000000                     0.000000   
25%                   0.345475                     0.376587   
50%                   0.469549                     0.498432   
75%                   0.583512

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (312). n_quantiles is set to n_samples.
  warnings.warn(


In [96]:
# Save the final standardized dataset
user_features.to_csv("Duolingo_data_03_06_final_standardized.csv", index=False)

In [97]:
df = pd.read_csv("Duolingo_data_03_06_final_standardized.csv")
pd.set_option('display.max_columns', None)
df

,user_id,block_average_correctness,reverse_tap_average_correctness,reverse_translate_average_correctness,listen_average_correctness,block_average_response_time,reverse_tap_average_response_time,reverse_translate_average_response_time,listen_average_response_time,user_ability_score,average_blocks_per_session,average_blocks_per_day,average_sessions_per_day,session_length_variance,blocks_per_session_variance,blocks_per_day_variance,sessions_per_day_variance,session_count,block_count,listen,reverse_tap,reverse_translate,response_time_variance,churn_time,churn_time_category
0,+4kwmfjD,0.227673,0.133858,0.378947,0.375000,0.002246,0.031466,0.003641,0.008832,0.610942,0.508028,0.514156,0.444444,0.755147,0.789055,0.000000,0.000000,0.020408,0.067494,0.045997,0.0768,0.047328,0.163987,0.0,1
1,+IUzKF2l,0.332900,0.212815,0.500000,0.607595,0.002769,0.033279,0.003321,0.011009,0.646135,0.177752,0.045300,0.000000,0.329076,0.356443,0.441670,0.000000,0.183673,0.103226,0.063032,0.1304,0.039695,0.237942,22.0,4
2,+Xv/6Nop,0.229710,0.140162,0.375000,0.397849,0.005831,0.044804,0.004645,0.015200,0.606883,0.300131,0.130238,0.133333,0.539885,0.622328,0.744112,0.225838,0.102041,0.102730,0.068143,0.1208,0.051908,0.784566,5.0,1
3,+mMyHsNU,0.239135,0.000000,0.250307,0.225892,0.004481,0.000000,0.001964,0.008600,0.613978,0.322248,0.283126,0.355556,0.775884,0.715246,0.800309,0.458726,0.224490,0.214392,0.373083,0.0000,0.352672,0.536977,6.0,1
4,+ziSnwWC,0.169845,0.065789,0.266409,0.328947,0.005608,0.024309,0.006240,0.010858,0.590596,0.069626,0.036523,0.166667,0.320580,0.380484,0.539567,0.252993,0.387755,0.098263,0.061329,0.0856,0.111450,0.495177,13.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,y6797eWc,0.076945,0.015957,0.202454,0.090909,0.002950,0.014475,0.005458,0.009800,0.545205,0.123050,0.107305,0.333333,0.280746,0.191319,0.583260,0.376852,0.163265,0.066501,0.049404,0.0672,0.059542,0.700965,3.0,1
308,yka558y8,0.144504,0.103175,0.166065,0.175000,0.021478,0.014423,0.010699,0.063518,0.559939,0.176606,0.133635,0.277778,0.522893,0.604499,0.863963,0.584944,0.142857,0.081886,0.061329,0.0440,0.140458,0.897106,4.0,1
309,z/C43yrd,0.069785,0.044248,0.175258,0.088889,0.004046,0.030523,0.008156,0.009240,0.536617,0.075279,0.049183,0.222222,0.377410,0.472344,0.635845,0.376852,0.244898,0.065509,0.039182,0.0800,0.041221,0.434084,6.0,1
310,z9FlLLzt,0.122254,0.000000,0.104911,0.105263,0.006673,0.000000,0.003753,0.007835,0.545946,0.159404,0.120895,0.277778,0.410431,0.406270,0.520763,0.085736,0.142857,0.074442,0.045997,0.0000,0.215267,0.356913,3.0,1
